### This Notebook consists of the following components:
1. import all packages, documents and merge into one df
2. label categories for strata selection
3. extract NL for each column and select the necessary columns
4. Sample and strata selection
5. For each strata sample, export the each file with id name, content of NL columns (with colums names as title of each bit) as conll files

### 1. Import package, documents and merge into a df

In [ ]:
import pandas as pd
import numpy as np
import glob
import re
from TextToCoNLL import text_to_conll
import spacy
import os

In [ ]:
nlp=spacy.load('nl_core_news_sm')

In [ ]:
pd.options.display.max_colwidth =5500

### 1.1 filter to 5 columns: "Identificatie","SCMcode","Toelichting","Bevinding","Verklaring"

In [ ]:
def read_and_filter(filename):
    """ read csv and filter out columns "Identificatie","Toelichting","Bevinding","Verklaring" """
    df = pd.read_csv(filename, delimiter=";")
    df = df.replace(np.nan, " ")
    
    cf = pd.read_csv("/data/cybercrime/data/Feitcodetabel.csv", delimiter=";") #can leave in the globe scope
    nf = cf.merge(df, how='inner', left_on= 'Feitcode', right_on="Code").drop('Feitcode', axis=1) #add column with SCMcode
    
    df = nf.filter(items=["Identificatie","SCMcode","Toelichting","Bevinding","Verklaring"])
    return df

### 1.2 merge all dfs into 1 df

In [ ]:
all_df=[]
for filename in glob.glob("/data/cybercrime/data/*2016-??.csv"):
    df=read_and_filter(filename)
    all_df.append(df)

In [ ]:
pf =  pd.concat([d.set_index("Identificatie") for d in all_df], axis=0, ignore_index=False).reset_index()

## 2. Add labels

### 2.1 define and add source labels

In [ ]:
def get_source_label(row):
    """"""
    verklaring=row["Verklaring"]
    if len(verklaring) < 2:
        return "p"
    if "LMIO" in row["Toelichting"] or "LMIO" in row["Verklaring"] : 
        return "L"
    else:
        return "nL"

In [ ]:
pf["Bron"] = pf.apply(lambda row: get_source_label(row), axis=1)

### 2.2 add SCM labels

In [ ]:
def get_SCM(row):
    SCM = row["SCMcode"]
    if SCM < 2000000:
        return "V"
    else:
        return "O"

In [ ]:
pf["SCM"] = pf.apply(lambda row: get_SCM(row), axis=1)

## 3. Extracting NL per section and label

### 3.1 Extract NL from Toelichting field

In [ ]:
def extract_nlt(row):
    "Extracts natural language texts from LMIO Toelichting section"
    l_start= "Via www.politie.nl is aangifte gedaan ter zake oplichting door de in deze registratie"
    end="(bereikbaar van maandag tot en met vrijdag tussen 08.00 en 17.00 uur).\r\n"
    p_start1="\n\nPlaats melder:"
    p_start2="\n\n\nINCIDENT: "
    t ="AANGIFTE OPGENOMEN MIDDELS INTERNET"
    t1="\n"
    t2="\n\n"
    t3="\n\n\n"
    t4="\n\n\n\n"
    ta="ER ZIJN REEDS GEGEVENS OVER HET VOORVAL DOORGEGEVEN"
    tb="POLITIE IS TER PLAATSE GEWEEST"
    tc="OOI\n"
    
    t0='In deze registratie gaat het om een melding tanken zonder betalen. Wanneer het kenteken drie maal bij de Landelijk Eenheid is geregistreerd voor tanken zonder betalen, dan worden de beschikbare gegevens van de Landelijke Eenheid doorgezonden naar de betrokken regio met het verzoek een opsporingsonderzoek naar de verantwoordelijke dader/kentekenhouder op te starten.\n\nVoor meer informatie neem contact op met de politie Landelijke Eenheid email: mailboxtankenzonderbetalen@klpd.politie.nl of op nummer 088-6628268 (ma t/m vr  08:00 tot 12:30 uur en van 13:00 tot 15:30 uur).\n\nVoor het opvragen van documenten (aangiften, beelden c.q. schuldbekentenissen en/of kopie ID-bewijzen) dient u zich te vervoegen bij de infodesk in uw eenheid. De infodesk legt dan het verzoek neer bij de infodesk van de Landelijke Eenheid. (mailboxdlioinformatieknooppuntle@klpd.politie.nl) \n'
    t00="In deze registratie gaat het om een melding tanken zonder betalen. Wanneer het kenteken drie maal bij de Landelijk Eenheid is geregistreerd voor tanken zonder betalen, dan worden de beschikbare gegevens van de Landelijke Eenheid doorgezonden naar de betrokken regio met het verzoek een opsporingsonderzoek naar de verantwoordelijke dader/kentekenhouder op te starten.\n\nVoor meer informatie neem contact op met de politie Landelijke Eenheid email: mailboxtankenzonderbetalen@klpd.politie.nl of op nummer 088-6628268 (ma t/m vr 08:00 tot 12:30 uur en van 13:00 tot 15:30 uur).\n\nVoor het opvragen van documenten (aangiften, beelden c.q. schuldbekentenissen en/of kopie ID-bewijzen) dient u zich te vervoegen bij de infodesk in uw eenheid. De infodesk legt dan het verzoek neer bij de infodesk van de Landelijke Eenheid. (mailboxdlioinformatieknooppuntle@klpd.politie.nl) \n"
    tl="Schade omschrijving LANTAARNPAAL: Lichtmast is aangereden door een onbekend motorvoertuig."
    tl2="AANGIFTE OPGENOMEN MIDDELS INTERNET\n\n\n\nSchade omschrijving LANTAARNPAAL: Er is schade veroorzaakt aan een lichtmast welk is aangereden door een onbekend motorvoertuig.\nIndien u informatie beschikt omtrent deze schade zouden wij graag het gemaakte proces verbaal met het verbaalnummer willen ontvangen via U of via SPV.\nEr is een buurtonderzoek opgestart om te achterhalen of er een dader bekend is."
    ti="\nDeze toelichting is door Aangifte via Intranet automatisch aangevuld.\nOorspronkelijke toelichting:"
    
    if l_start in row: #1
        row=row.split(l_start)[0] #extract text before standard line
        if row == t or row == t+t1 or row == t+t2 or row == t+t3 or row == t+t4+tl or row == tc+t+t3 or row == tc+t1+t+t1 or row == tc+t1+t+t3 or row == t+t1+ta or row == t+t2+ta or row == t+t3+ta or row == t+t4+ta or row == t+t4+ta+t1+tb:
            text= row.replace(row," ")
            return text
        else:
            text=row
            return text
    elif p_start1 in row:#2
        text=row.split(p_start1)[0]
        return text
    elif p_start2 in row:#3
        text=row.split(p_start2)[0]
        return text
    elif row == t or row == t+t1 or row == t+t2 or row == t+t3 or row == t+t4+tl or row == tc+t+t3 or row == tc+t1+t+t1 or row == tc+t1+t+t3 or row == t+t1+ta or row == t+t2+ta or row == t+t3+ta or row == t+t4+ta or row == t+t4+ta+t1+tb:
        text= row.replace(row," ") #4
        return text
    elif row == t1 or row == ti or row == tl2: #5
        text= row.replace(row," ")
        return text
    elif row == t0 or row == t00: #6 tank zonder betalen
        text= row.replace(row," ")
        return text
    else:
        return row

### 3.2 Extract NL from Verklaring field

In [ ]:
def extract_nlv(row):
    "Extracts natural language texts from LMIO Verklaring section"
    start="\nConflict Omschrijving conflict"
    end="\nConflict"
    l1="De lichtmast is aangereden door een onbekend gebleven motorrijtuig.\n\n\n"
    l2="Lichtmast is aangereden door onbekend motorvoertuig.\n\n\n"
    l3="Lichtmast aangereden door onbekend motorvoertuig.\n\n\n"
    l4="Lichtmast aangereden door onbekend motorvoertuig\n\n\n"
    l5="Lichtmast is aangereden door een onbekend motorvoertuig.\n\n\n"
    l6="Er is schade veroorzaakt aan een lichtmast welk is aangereden door een onbekend motorvoertuig.\nIndien u informatie beschikt omtrent deze schade zouden wij graag het gemaakte proces verbaal met het verbaalnummer willen ontvangen via U of via SPV.\nEr is een buurtonderzoek opgestart om te achterhalen of er een dader bekend is.\n\n\n"
    l7="Door een onbekend gebleven motorvoertuig is een\nlichtmast aangereden en deze heeft hierdoor schade\nopgelopen. Er is een buurtonderzoek opgestart om te\nkijken of er een dader bekend is. Als u over informatie\nbeschikt aangaande deze schade, zouden wij graag in\nhet bezit komen van het gemaakte proces verbaal via u\nof via SPV, graag het verbaalnummer doormailen.\n\n\n"
    l8="Lichtmast aangereden door onbekend motorvoertuig."
    l9="De lichtmast is aangereden door een onbekend gebleven motorrijtuig."
    
    c0="Ik ben ervan op de hoogte dat camerabeelden 12 maanden bewaard moeten blijven."
    c1="Ik ben ervan op de hoogte dat camerabeelden 12 maanden bewaard moeten blijven.\n\t\n"
    
    f0="Fiets gestolen.\n\n\n"
    f1="Mijn fiets is gestolen.\n\n\n"
    f2="Fiets gestolen\n\n\n"
    f3='Mijn fiets is gestolen\n\n\n'
    f4="fiets gestolen\n\n\n"
    f5="Ik ben eigenaar van genoemde fiets. \r\nOp eerst genoemde dag, datum en tijdstip heb ik mijn fiets geplaatst op genoemde lokatie. Mijn fiets was terdege afgesloten. \r\nToen ik op laatst genoemde dag, datum en tijdstip mijn fiets wilde gebruiken, zag ik dat mijn fiets door onbekende(n) was weggenomen."
    
    if start in row:
        part=row.split(start)[1]
        text=part.split(end)[0]
        return text
    elif "AANGIFTE OPGENOMEN MIDDELS INTERNET\n\n\n" in row: #vaak voorkomend standaard textje
        text = row.replace("AANGIFTE OPGENOMEN MIDDELS INTERNET\n\n\n"," ")
        return text
    elif row == l1 or row == l2 or row == l3 or row == l4 or row == l5 or row == l6 or row == l7 or row == l8 or row == l9 or row == c0 or row == c1 or row == f0 or row == f1 or row == f2 or row == f3 or row == f4 or row == f5:
        text = row.replace(row," ")
        return text
    else:
        return row

In [ ]:
pf["NLT"] = pf.apply(lambda row: extract_nlt(row["Toelichting"]), axis=1)
pf["NLV"] = pf.apply(lambda row: extract_nlv(row["Verklaring"]), axis=1)

### 3.2 Check which rows contains text by adding label

In [ ]:
pf = pf.replace(np.nan, " ") #replace nans

In [ ]:
def contains_text(row):
    if len(row) > 2:
        return "text"
    else:
        return "no text"

In [ ]:
pf["TEXT_T"] = pf.apply(lambda row: contains_text(row["NLT"]), axis=1)
pf["TEXT_V"] = pf.apply(lambda row: contains_text(row["NLV"]), axis=1)

In [ ]:
ff = pf.filter(items=["Identificatie","NLT","NLV","SCM","Bron","Toelichting","Verklaring","TEXT_T","TEXT_V"])

In [ ]:
ff.head(1)

In [ ]:
ff.describe(include="all")

In [ ]:
pf.groupby(["Bron","SCM","TEXT_V"]).size() #check how many text per strata exist

In [ ]:
pf.groupby(["Bron","SCM","TEXT_T"]).size()

### 4 Sample section

### 4.1 Strata selection function

In [ ]:
def select_samp(source,veld,scm):
    """
    :param param1: name source string "LMIO","non-LMIO" or "politie"
    :param param2: ẗoelichting veld: "TEXT_T" or verklaring veld "TEXT_V"
    :param param3: scm: "Vermogen" or Overig"
    """
    s = ff.loc[(ff["Bron"]==source)&(ff[veld]=="text")&(ff["SCM"]==scm)]
    if veld == "TEXT_T":
        selection = s.filter(items=["Identificatie","NLT","SCM","Bron"])
    else:
        selection = s.filter(items=["Identificatie","NLV","SCM","Bron"])
    return selection

### 4.2 Select strata

In [ ]:
LtO=select_samp("L","TEXT_T", "O")
LvO=select_samp("L","TEXT_V", "O")
LtV=select_samp("L","TEXT_T", "V")
LvV=select_samp("L","TEXT_V", "V")

nLtO=select_samp("nL","TEXT_T", "O")
nLvO=select_samp("nL","TEXT_V", "O")
nLtV=select_samp("nL","TEXT_T", "V")
nLvV=select_samp("nL","TEXT_V", "V")

ptO=select_samp("p","TEXT_T", "O")
ptV=select_samp("p","TEXT_T", "V")

### 4.3 Set random seed

In [ ]:
seed = 98527684 #952768498: 305 952728498:389 952728498 972728498

### 4.4 Select samples per strata

In [ ]:
def sample_55(df): 
    sf = df.sample(n = 60, replace = False, random_state = rng)
    return sf

In [ ]:
def sample_12(df):
    sf = df.sample(n = 12, replace = False, random_state = rng)
    return sf

In [ ]:
def sample_98(df):
    sf = df.sample(n = 105, replace = False, random_state = rng)
    return sf

In [ ]:
def sample_125(df):
    sf = df.sample(n = 125, replace = False, random_state = rng)
    return sf

In [ ]:
#ronde 1
rng = np.random.RandomState(seed) #set only once; not for every round
sLtO = sample_12(LtO)
sLvO = sample_12(LvO)

sLtV = sample_98(LtV)
sLvV = sample_98(LvV)

snLvO = sample_55(nLvO)
snLvV = sample_55(nLvV)

snLtO = sample_55(nLtO)
snLtV = sample_55(nLtV)

sptO = sample_55(ptO)
sptV = sample_55(ptV)

In [ ]:
#ronde 2
sLtV = sample_125(LtV)
sLvV = sample_125(LvV)

snLvO = sample_55(nLvO)
snLvV = sample_55(nLvV)

snLtO = sample_55(nLtO)
snLtV = sample_55(nLtV)

sptO = sample_55(ptO)
sptV = sample_55(ptV)

In [ ]:
#ronde 1
smp = [sLtO, sLvO, sLtV, sLvV, snLvV, snLtO, sptO, sptV, snLvO, snLtV] # strata
samplesize = [12,12, 98, 98, 55,55,55, 55,55, 55]# number of texts to select

In [ ]:
#ronde 2
smp = [sLtV, sLvV, snLvV, snLtO, sptO, sptV, snLvO, snLtV] # strata
samplesize = [120, 120, 55,55,55, 55,55, 55] # number of texts to select

### 5 Take strata selections and samplesizes and output conll files per sta

In [ ]:
for df,sz in zip(smp,samplesize):
    lf = list(df)
    if "NLT" in lf:
        col = list(df["NLT"])
        field ="t"
    if "NLV" in lf:
        col = list(df["NLV"])
        field ="v"

    IDs = list(df["Identificatie"])
    sources = list(df["Bron"])
    scm = list(df["SCM"])
    nlist = list(range(1,125))

    teller=0
    for n,txt,ID,s,c in zip(nlist,col,IDs,sources,scm):
        try:
            text_to_conll(text=txt,
                        nlp=nlp,
                        delimiter=" ",
                        output_dir='sample_2',
                        basename=f'{s}{field}{c}_{n}_{ID}.conll',
                        spacy_attrs=['text', 'lemma_', 'ent_type_', 'ent_type_'],
                        default_values={'ent_type_': "O"},
                        exclude=['is_space'],
                        start_with_index=False,
                        verbose=1,
                        overwrite_existing_conll_file=True)
            teller=teller+1
            if teller == sz:
                break
        except AssertionError:
            print(f"fout in conll file {s}{field}{c}_{n}_{ID}.conll")
            os.remove(f"sample_2/{s}{field}{c}_{n}_{ID}.conll")